In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")
df

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id,target
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-01-29,4074.14,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-01-30,4074.14,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-01-31,4074.14,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-02-01,4074.14,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-02-02,4074.14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69285,0,0,0,0,0,0,0,0,0,0,...,4,0,1,3,0,0,1,2024-12-27,JC116,0
69286,0,0,0,0,0,0,0,0,2,0,...,0,1,2,0,6,2,1,2024-12-28,JC116,0
69287,0,0,0,0,0,1,0,0,2,0,...,1,1,1,0,0,0,0,2024-12-29,JC116,3
69288,0,0,0,0,0,0,2,0,0,0,...,2,1,0,1,3,1,2,2024-12-30,JC116,1


In [4]:
from datetime import datetime
from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2024, 9, 1, 0, 0, 0),  # ✅ Updated to 2024
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(44280, 674)
(44280,)
(25010, 674)
(25010,)


In [5]:
X_train

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-01-29,4074.14
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-01-30,4074.14
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-01-31,4074.14
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-02-01,4074.14
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-02-02,4074.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44275,1,1,0,0,0,0,0,1,6,1,...,8,12,12,5,7,2,1,3,2024-08-27,JC116
44276,2,0,0,0,0,0,0,2,2,2,...,10,21,7,7,8,1,1,0,2024-08-28,JC116
44277,0,0,0,0,0,0,0,2,5,2,...,4,16,10,10,5,2,0,0,2024-08-29,JC116
44278,1,0,0,0,0,0,1,0,0,2,...,7,12,12,6,2,3,1,1,2024-08-30,JC116


In [6]:
past_ride_columns = [c for c in X_train.columns if c.startswith("rides_")]
X_train_only_numeric = X_train[past_ride_columns]
X_test_only_numeric = X_test[past_ride_columns]


In [7]:
import xgboost as xgb
model = xgb.XGBRegressor(max_depth=5)
model.fit(X_train_only_numeric, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [8]:
from sklearn.metrics import mean_absolute_error
predictions = model.predict(X_test_only_numeric)

In [9]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

0.2023


In [10]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
import os
load_dotenv() 

mlflow = set_mlflow_tracking()
#log_model_to_mlflow(model, X_test_only_numeric, "XGBoost", "mean_absolute_error", score=test_mae)
log_model_to_mlflow(model, X_test_only_numeric.sample(1000, random_state=42), "XGBoost", "mean_absolute_error", score=test_mae)


INFO:src.experiment_utils:MLflow tracking URI and credentials set.
2025/05/05 23:02:27 INFO mlflow.tracking.fluent: Experiment with name 'XGBoost' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: XGBoost
INFO:src.experiment_utils:Logged mean_absolute_error: 0.20228145442320988
C:\Users\vasub\anaconda3\envs\citi_bike25\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest

2025/05/05 23:02:39 INFO mlflow.models.model: Found the following environment variables used during model inference: [HOPSWORKS_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
Registered model 'citi_bike_ride_predictor_next_hour' already exists. Creating a new version of this model...
2025/05/05 23:02:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: citi_bike_ride_predictor_next_hour, version 4
Created version '4' of model 'citi_bike_ride_predictor_next_hour'.
INFO:src.experiment_utils:Model logged with name: citi_bike_ride_predictor_next_hour


🏃 View run nervous-ram-617 at: https://dagshub.com/vasukrishna001/cityBikes25_rides.mlflow/#/experiments/3/runs/040250ed26f14810adc7ca4fac1241c0
🧪 View experiment at: https://dagshub.com/vasukrishna001/cityBikes25_rides.mlflow/#/experiments/3
